In [1]:
from create_word_vectors import * 
import pickle 
import helper
from tfidf_methods import * 
from paths import * 
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import GlobalAveragePooling1D
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from keras.preprocessing import sequence

Using TensorFlow backend.


In [ ]:
# Prepare glove embedding mazrix and save it in embeddings
X_sequences, y, Kgl_sequences, nb_word , word_embedding, glove_matrix = run_glove_embbedding(GLOVE_EMBEDDING
                                        + "embeddings.pkl", n_grams = 1, pretrained = True, max_words = None)

In [2]:
# read X_train, y, Kgl_seuquences, nb_word , word_embedding, glove_matrix from the glove file
[X_sequences, y, Kgl_sequences, nb_word , word_embedding, glove_matrix] = pickle.load(open(GLOVE_EMBEDDING + "emebddings.pkl", "rb"))

In [3]:
from run_models import *

In [ ]:
# save the following values to train models on google colab cluster
#pickle.dump([X_sequences, y, Kgl_sequences,nb_word, glove_matrix], open('cluster_2ngrams.pkl', 'wb'),protocol=2)


'''
import gzip
import pickle
fp=gzip.open('cluster.pkl.gz','rb')
[X_sequences, y, Kgl_sequences,nb_word, glove_matrix]=pickle.load(fp)

import gzip
import pickle
fp=gzip.open('cluster.pkl.gz','rb')
[X_sequences, y, Kgl_sequences,nb_word, glove_matrix]=pickle.load(fp)
'''

In [ ]:
# Read cleaned files! we will not always perform 
X = pickle.load(open(CLEANED_DATA_PATH + 'X_Cleaned.pkl', "rb"))
X_Kgl = pickle.load(open(CLEANED_DATA_PATH + 'X_Kgl_Cleaned.pkl', "rb"))

In [ ]:
# Prepare tfidf embedding
#X_train_tfidf_avg, X_test_tfidf_avg, X_train_tfidf, X_test_tfidf = tfidf_embedding(X, X_Kgl, word_embedding)

In [ ]:
X_sequences.shape

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=200))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train_tfidf_avg, y, 32,epochs=2)

In [ ]:
DATA_MODELS = 'models_saving/'

In [ ]:
# Fisrt model
np.random.seed(1)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)
train_model_1 = model.predict_proba(X_sequences, batch_size=128)
test_model_1 = model.predict_proba(Kgl_sequences)

pickle.dump(train_model_1, open(DATA_MODELS + 'train_model_1', 'wb'))
pickle.dump(test_model_1, open(DATA_MODELS + 'test_model_1', 'wb'))

In [ ]:
# second model: Embedding + Convolution + MaxPooling+LSTM+ Dense layer with sigmoid activation
np.random.seed(2)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)
train_model_2 = model.predict_proba(X_sequences, batch_size=128)
test_model_2 = model.predict_proba(Kgl_sequences)

pickle.dump(train_model_2, open(DATA_MODELS + 'train_model_2', 'wb'))
pickle.dump(test_model_2, open(DATA_MODELS + 'test_model_2', 'wb'))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# third model

np.random.seed(3)

model = Sequential()
model.add(Embedding(nb_word+1,200, input_length = X_sequences.shape[1], weights=[glove_matrix]))
model.add(SpatialDropout1D(0.4))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)
train_model_3 = model.predict_proba(X_sequences, batch_size=128)
test_model_3 = model.predict_proba(Kgl_sequences)

pickle.dump(train_model_3, open(DATA_MODELS + 'train_model_3', 'wb'))
pickle.dump(test_model_3, open(DATA_MODELS + 'test_model_3', 'wb'))

In [ ]:
np.random.seed(4)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(LSTM(100, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)

train_4 = model.predict_proba(X_sequences, batch_size=128)
test_4 = model.predict_proba(Kgl_sequences)

pickle.dump(train_4, open(DATA_MODELS + 'train_model_4', 'wb'))
pickle.dump(test_4, open(DATA_MODELS + 'test_model_4', 'wb'))

In [ ]:
np.random.seed(5)

model = Sequential()
model.add(Embedding(1, 200, input_length=X_sequences.shape[1]))
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1)

train_5 = model.predict_proba(X_sequences, batch_size=128)
test_5 = model.predict_proba(Kgl_sequences)

pickle.dump(train_5, open(DATA_MODELS + 'train_model_5', 'wb'))
pickle.dump(test_5, open(DATA_MODELS + 'test_model_5', 'wb'))

In [ ]:
np.random.seed(6)

model = Sequential()
model.add(Embedding(nb_word+1, 50, input_length=X_sequences.shape[1]))
model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)

train_6 = model.predict_proba(X_sequences, batch_size=128)
test_6 = model.predict_proba(Kgl_sequences)

pickle.dump(train_6, open(DATA_MODELS + 'train_model_6', 'wb'))
pickle.dump(test_6, open(DATA_MODELS + 'test_model_6', 'wb'))

In [ ]:
np.random.seed(7)

model = Sequential()
model.add(Embedding(nb_word+1, 50, input_length=X_sequences.shape[1]))
model.add(Convolution1D(filters=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)

train_7 = model.predict_proba(X_sequences, batch_size=128)
test_7 = model.predict_proba(Kgl_sequences)

pickle.dump(train_7, open(DATA_MODELS + 'train_model_7', 'wb'))
pickle.dump(test_7, open(DATA_MODELS + 'test_model_7', 'wb'))

In [ ]:
np.random.seed(8)

model = Sequential()
model.add(Embedding(nb_word+1, 50, input_length=X_sequences.shape[1]))
model.add(Convolution1D(filters=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_length=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)

train_8 = model.predict_proba(X_sequences, batch_size=128)
test_8 = model.predict_proba(Kgl_sequences)

pickle.dump(train_8,  open(DATA_MODELS + 'train_model_8', 'wb'))
pickle.dump(test_8, open(DATA_MODELS + 'test_model_8', 'wb'))

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train_tfidf_avg, y, epochs=3, batch_size=32, verbose=2)

In [ ]:
# Fisrt model
np.random.seed(9)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)
train_model_9 = model.predict_proba(X_sequences, batch_size=128)
test_model_9 = model.predict_proba(Kgl_sequences)

pickle.dump(train_model_9, open(DATA_MODELS + 'train_model_9', 'wb'))
pickle.dump(test_model_9, open(DATA_MODELS + 'test_model_9', 'wb'))

In [ ]:
# Fisrt model
np.random.seed(10)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(Convolution1D(nb_filter=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)
train_model_10 = model.predict_proba(X_sequences, batch_size=128)
test_model_10 = model.predict_proba(Kgl_sequences)

pickle.dump(train_model_10, open(DATA_MODELS + 'train_model_10', 'wb'))
pickle.dump(test_model_10, open(DATA_MODELS + 'test_model_10', 'wb'))

In [ ]:
from keras.layers import Dense, GRU
np.random.seed(11)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(GRU(units=16, name = "gru_1",return_sequences=True))
model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
model.add(GRU(units=4, name= "gru_3"))
model.add(Dense(1, activation='sigmoid',name="dense_1"))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(X_sequences, y, validation_split=0.1, epochs=1, batch_size=128)

train_11 = model.predict_proba(X_sequences, batch_size=128)
test_11 = model.predict_proba(Kgl_sequences)

pickle.dump(train_11,  open(DATA_MODELS + 'train_model_11', 'wb'))
pickle.dump(test_11, open(DATA_MODELS + 'test_model_11', 'wb'))

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

np.random.seed(12)

model = Sequential()
model.add(Embedding(nb_word+1, 200, input_length=X_sequences.shape[1], weights=[glove_matrix]))
model.add(GRU(100, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)

train_12 = model.predict_proba(X_sequences, batch_size=128)
test_12 = model.predict_proba(Kgl_sequences)

pickle.dump(train_12, open(DATA_MODELS + 'train_model_12', 'wb'))
pickle.dump(test_12, open(DATA_MODELS + 'test_model_12', 'wb'))

In [ ]:
np.random.seed(13)

model = Sequential()
model.add(Embedding(nb_word+1, 50, input_length=X_sequences.shape[1]))
model.add(Convolution1D(filters=32, filter_length=3, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_length=2))
model.add(GRU(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_sequences, y, validation_split=0.1, epochs=2, batch_size=128, verbose=1, shuffle=True)

train_13 = model.predict_proba(X_sequences, batch_size=128)
test_13 = model.predict_proba(Kgl_sequences)

pickle.dump(train_13,  open(DATA_MODELS + 'train_model_13', 'wb'))
pickle.dump(test_13, open(DATA_MODELS + 'test_model_13', 'wb'))